In [101]:
require('nn')
require('hdf5')
require('optim')

In [102]:
f = hdf5.open("data.hdf5", "r")
X_train = f:read("train_input"):all()
Y_train = f:read("train_output"):all()
X_valid = f:read("valid_input"):all()
q_valid = f:read("valid_q"):all()
X_test = f:read("test_input"):all()
q_test = f:read("test_q"):all()
kaggle_valid = f:read("valid_kaggle"):all()
nclasses = f:read('nclasses'):all():long()[1]
nfeatures = f:read('nfeatures'):all():long()[1]
f:close()

window_size = X_train:size(2)
n_actual = nfeatures

In [105]:
--changes index to account for window size
function changeIndex(X, window_size, nfeatures)
    for i=1,X:size(2) do
        X:select(2,i):add((i-1)*nfeatures)
    end
    return X
end

function lookup(X, Y, win_size, C, K)
    W = torch.zeros(C, win_size*(K))
    for i=1,X:size(1) do
        for j=1,X:size(2) do
            local row = Y[i]
            local col = X[i][j]
            W[row][col] = W[row][col] + 1
        end
    end
    return W
end

In [115]:
function CBM(fc, Y, alpha, V)
    --calculate log posterior
    local cc = torch.zeros(1, V)
    local clp = torch.zeros(1, V)

    --collapse column wise so Nx1
    local hist = torch.histc(Y:double(), 10001)
    cc:add(torch.reshape(hist, 1, 10001))
    --THIS IS ALPHA
    --smoothing
    fc:add(alpha)
    local scc = fc:sum(2)
    
    fc:log()
    scc:log()
    
    scc:expand(scc, scc:size(1), fc:size(2))
    
    --calculate log prior
    --collapse down so 1x1
    local total = cc:sum(2)
    cc:div(total[1][1])
    cc:log()
    --clp:csub(math.log(Y:size(2)))
    
    return fc:csub(scc), cc
end

function predict(X, W, b)
  local jll = torch.zeros(X:size(1), W:size(1))
  for i = 1, X:size(1) do
    jll[i] = W:index(2, X[i]):sum(2)
    --jll[i]:add(b)
  end
  return jll
end

In [107]:
function perplexity(jll, Y)
    local matrix = torch.zeros(Y:size(1), Y:size(2))
    for i=1, Y:size(1) do
        for j=1, Y:size(2) do
            idx = Y[i][j]
            matrix[i][j] = (jll[i][idx] * -1)
        end
    end
    return matrix:cdiv(torch.expand((torch.sum(matrix, 2)), Y:size(1), matrix:size(2)))
end

function write2file(scores, fname)
    f = io.open(fname, "w")
    f:write("ID,Class1,CLass2,Class3,Class4,Class5,Class6,Class7,Class8,Class9,Class10,Class11,Class12,CLass13,Class14,Class15,Class16,Class17,Class18,Class19,Class20,Class21,Class22,Class23,CLass24,Class25,Class26,Class27,Class28,Class29,Class30,Class31,Class32,Class33,Class34,CLass35,Class36,Class37,Class38,Class39,Class40,Class41,Class42,Class43,Class44,Class45,CLass46,Class47,Class48,Class49,Class50\n")
    for i=1,scores:size(1) do
        s = tostring(i)
        for j=1, scores:size(2) do
            s = s .. "," .. tostring(scores[i][j])
        end
        f:write(s .. "\n")
    end
    f:close()
end

In [108]:
X_train = changeIndex(X_train, 5, nfeatures)
X_valid = changeIndex(X_valid, 5, nfeatures)
X_test = changeIndex(X_test, 5, nfeatures)

In [109]:
W = lookup(X_train, Y_train, window_size, 10001, nfeatures)

In [110]:
W_, b = CBM(W, Y_train, 1, 10001)

In [116]:
jll = predict(X_test, W_, b)

In [117]:
scores = perplexity(jll, q_test)

In [119]:
write2file(scores, "pred_2.csv")

In [114]:
b

Columns 1 to 6
 -3.0199e+00  -3.0295e+00         -inf  -3.0038e+00  -3.2760e+00  -3.5749e+00

Columns 7 to 12
 -3.5934e+00  -3.7611e+00  -3.9797e+00  -3.9263e+00  -4.5794e+00  -4.6263e+00

Columns 13 to 18
 -4.6289e+00  -4.7374e+00  -4.8593e+00  -5.1906e+00  -5.0494e+00  -5.0703e+00

Columns 19 to 24
 -5.2017e+00  -5.2485e+00  -5.2610e+00  -5.2114e+00  -5.2101e+00  -5.2720e+00

Columns 25 to 30
 -5.7459e+00  -5.4356e+00  -5.3758e+00  -5.4786e+00  -5.4380e+00  -5.8316e+00

Columns 31 to 36
 -6.1918e+00  -5.5755e+00  -5.5733e+00  -5.5241e+00  -5.6085e+00  -5.6224e+00

Columns 37 to 42
 -5.8357e+00  -5.7612e+00  -5.9276e+00  -6.0886e+00  -6.1204e+00  -5.9338e+00

Columns 43 to 48
 -5.9254e+00  -5.9698e+00  -5.9842e+00  -6.0623e+00  -6.0695e+00  -6.1182e+00

Columns 49 to 54
 -6.0865e+00  -6.1097e+00  -6.2391e+00  -6.1831e+00  -6.1740e+00  -6.1700e+00

Columns 55 to 60
 -6.2630e+00  -6.4200e+00  -6.2128e+00  -6.2519e+00  -6.5468e+00  -6.2294e+00

Columns 61 to 66
 -6.5191e+00  -6.3258e+00 

58
 -7.7368e+00  -7.7022e+00  -7.7423e+00  -7.7048e+00  -7.7154e+00  -7.9306e+00

Columns 259 to 264
 -7.7234e+00  -7.7840e+00  -7.7477e+00  -7.7926e+00  -7.8485e+00  -7.7477e+00

Columns 265 to 270
 -7.7727e+00  -7.8577e+00  -7.7840e+00  -7.8951e+00  -7.7671e+00  -7.8762e+00

Columns 271 to 276
 -7.7615e+00  -7.7954e+00  -7.9472e+00  -7.7868e+00  -7.7897e+00  -7.8041e+00

Columns 277 to 282
 -7.7868e+00  -7.8012e+00  -8.1302e+00  -7.8335e+00  -7.8187e+00  -8.0831e+00

Columns 283 to 288
 -8.0455e+00  -7.9405e+00  -7.8275e+00  -8.0093e+00  -7.8856e+00  -7.9078e+00

Columns 289 to 294
 -7.9143e+00  -7.8246e+00  -7.8070e+00  -7.9916e+00  -8.0200e+00  -7.9046e+00

Columns 295 to 300
 -7.9743e+00  -7.9078e+00  -7.9372e+00  -7.8638e+00  -7.8824e+00  -7.9111e+00

Columns 301 to 306
 -7.9273e+00  -8.0382e+00  -7.8335e+00  -7.9143e+00  -8.0492e+00  -7.9014e+00

Columns 307 to 312
 -7.8951e+00  -7.8762e+00  -7.8700e+00  -7.9143e+00  -7.8669e+00  -7.9372e+00

Columns 313 to 318
 -7.9438e+00  -7.

00

Columns 505 to 510
 -8.3344e+00  -8.3053e+00  -8.3644e+00  -8.3053e+00  -8.2910e+00  -8.3394e+00

Columns 511 to 516
 -8.3798e+00  -8.3005e+00  -8.3394e+00  -8.5356e+00  -8.3543e+00  -8.3594e+00

Columns 517 to 522
 -8.3198e+00  -8.3246e+00  -8.3901e+00  -8.4112e+00  -8.6640e+00  -8.3344e+00

Columns 523 to 528
 -8.5296e+00  -8.3246e+00  -8.5001e+00  -8.4006e+00  -8.3543e+00  -8.4828e+00

Columns 529 to 534
 -8.3344e+00  -8.4112e+00  -8.4658e+00  -8.4165e+00  -8.3594e+00  -8.3443e+00

Columns 535 to 540
 -8.3344e+00  -8.3746e+00  -8.4219e+00  -8.4059e+00  -8.4273e+00  -8.4828e+00

Columns 541 to 546
 -8.3695e+00  -8.3644e+00  -8.3901e+00  -8.3644e+00  -8.3849e+00  -8.4112e+00

Columns 547 to 552
 -8.3849e+00  -8.4006e+00  -8.3746e+00  -8.4381e+00  -8.4327e+00  -8.4491e+00

Columns 553 to 558
 -8.4165e+00  -8.5725e+00  -8.4059e+00  -8.4491e+00  -8.4273e+00  -8.4658e+00

Columns 559 to 564
 -8.4658e+00  -8.5356e+00  -8.6107e+00  -8.4602e+00  -8.5662e+00  -8.5850e+00

Columns 565 to 5

.7132e+00  -8.7424e+00

Columns 757 to 762
 -8.7276e+00  -8.6918e+00  -8.7498e+00  -8.7573e+00  -8.7498e+00  -8.8273e+00

Columns 763 to 768
 -8.7204e+00  -9.0938e+00  -8.7204e+00  -8.8601e+00  -8.7060e+00  -8.8273e+00

Columns 769 to 774
 -8.8034e+00  -8.7956e+00  -8.8193e+00  -8.8034e+00  -8.7132e+00  -8.9027e+00

Columns 775 to 780
 -8.8940e+00  -8.7350e+00  -8.7498e+00  -8.7350e+00  -8.7424e+00  -8.9381e+00

Columns 781 to 786
 -8.7724e+00  -8.8354e+00  -9.0032e+00  -8.7424e+00  -8.8273e+00  -8.7498e+00

Columns 787 to 792
 -8.7424e+00  -8.9655e+00  -8.7424e+00  -8.7801e+00  -8.7276e+00  -8.7724e+00

Columns 793 to 798
 -8.9842e+00  -8.7424e+00  -8.7878e+00  -8.7498e+00  -8.7724e+00  -8.8193e+00

Columns 799 to 804
 -9.1478e+00  -8.7724e+00  -8.8685e+00  -8.8769e+00  -8.8113e+00  -8.8518e+00

Columns 805 to 810
 -8.7801e+00  -8.7878e+00  -8.8113e+00  -8.8436e+00  -8.8034e+00  -8.9563e+00

Columns 811 to 816
 -8.7956e+00  -8.7573e+00  -8.7878e+00  -8.7801e+00  -9.0325e+00  -8.8436e+

37e+00  -8.9748e+00  -9.1590e+00  -9.0129e+00

Columns 1009 to 1014
 -9.0325e+00  -9.0032e+00  -1.0092e+01  -9.1590e+00  -8.9655e+00  -9.0627e+00

Columns 1015 to 1020
 -9.0129e+00  -9.0032e+00  -9.0938e+00  -9.0729e+00  -9.1368e+00  -8.9842e+00

Columns 1021 to 1026
 -9.1258e+00  -9.0032e+00  -9.2909e+00  -9.3038e+00  -8.9842e+00  -9.1150e+00

Columns 1027 to 1032
 -9.0627e+00  -9.0425e+00  -9.0129e+00  -9.0425e+00  -9.0729e+00  -9.1590e+00

Columns 1033 to 1038
 -9.0129e+00  -9.1368e+00  -9.0938e+00  -9.0525e+00  -8.9937e+00  -9.0938e+00

Columns 1039 to 1044
 -9.0525e+00  -9.0525e+00  -9.1933e+00  -9.0032e+00  -9.1150e+00  -9.0627e+00

Columns 1045 to 1050
 -9.3436e+00  -9.1703e+00  -9.2288e+00  -9.0627e+00  -9.0729e+00  -9.2656e+00

Columns 1051 to 1056
 -9.0833e+00  -9.0129e+00  -9.0227e+00  -9.0525e+00  -9.0227e+00  -9.1043e+00

Columns 1057 to 1062
 -9.0325e+00  -9.0525e+00  -9.0425e+00  -9.0938e+00  -9.0627e+00  -9.2050e+00

Columns 1063 to 1068
 -9.0525e+00  -9.0525e+00  -9.08

...

1856e+01  -1.2261e+01  -1.1856e+01  -1.2038e+01  -1.1856e+01  -1.2549e+01

Columns 8875 to 8880
 -1.2038e+01  -1.2261e+01  -1.2038e+01  -1.2038e+01  -1.1856e+01  -1.1856e+01

Columns 8881 to 8886
 -1.2261e+01  -1.1856e+01  -1.1856e+01  -1.1856e+01  -1.2038e+01  -1.2038e+01

Columns 8887 to 8892
 -1.1856e+01  -1.1856e+01  -1.1856e+01  -1.1856e+01  -1.1856e+01  -1.1856e+01

Columns 8893 to 8898
 -1.1856e+01  -1.2038e+01  -1.1856e+01  -1.2261e+01  -1.1856e+01  -1.1856e+01

Columns 8899 to 8904
 -1.2038e+01  -1.2038e+01  -1.1856e+01  -1.1856e+01  -1.2549e+01  -1.1856e+01

Columns 8905 to 8910
 -1.2038e+01  -1.2261e+01  -1.1856e+01  -1.1856e+01  -1.2261e+01  -1.1856e+01

Columns 8911 to 8916
 -1.1856e+01  -1.1856e+01  -1.2038e+01  -1.1856e+01  -1.1856e+01  -1.1856e+01

Columns 8917 to 8922
 -1.1856e+01  -1.2038e+01  -1.1856e+01  -1.1856e+01  -1.1856e+01  -1.2038e+01

Columns 8923 to 8928
 -1.2549e+01  -1.1856e+01  -1.1856e+01  -1.1856e+01  -1.1856e+01  -1.1856e+01

Columns 8929 to 8934
 -1.

 -1.1856e+01  -1.2261e+01  -1.2038e+01  -1.1856e+01  -1.1856e+01  -1.1856e+01

Columns 9121 to 9126
 -1.1856e+01  -1.1856e+01  -1.2038e+01  -1.1856e+01  -1.2038e+01  -1.1856e+01

Columns 9127 to 9132
 -1.1856e+01  -1.1856e+01  -1.2038e+01  -1.2038e+01  -1.1856e+01  -1.1856e+01

Columns 9133 to 9138
 -1.2038e+01  -1.1856e+01  -1.2038e+01  -1.1856e+01  -1.1856e+01  -1.1856e+01

Columns 9139 to 9144
 -1.1856e+01  -1.1856e+01  -1.1856e+01  -1.1856e+01  -1.2261e+01  -1.1856e+01

Columns 9145 to 9150
 -1.2038e+01  -1.2261e+01  -1.2038e+01  -1.2038e+01  -1.1856e+01  -1.1856e+01

Columns 9151 to 9156
 -1.1856e+01  -1.1856e+01  -1.1856e+01  -1.2261e+01  -1.2038e+01  -1.2261e+01

Columns 9157 to 9162
 -1.2261e+01  -1.1856e+01  -1.1856e+01  -1.1856e+01  -1.2261e+01  -1.1856e+01

Columns 9163 to 9168
 -1.1856e+01  -1.2261e+01  -1.1856e+01  -1.2549e+01  -1.2261e+01  -1.2038e+01

Columns 9169 to 9174
 -1.1856e+01  -1.2954e+01  -1.1856e+01  -1.1856e+01  -1.1856e+01  -1.2038e+01

Columns 9175 to 9180


366
 -1.2038e+01  -1.2038e+01  -1.1856e+01  -1.2038e+01  -1.1856e+01  -1.2038e+01

Columns 9367 to 9372
 -1.1856e+01  -1.2038e+01  -1.2038e+01  -1.1856e+01  -1.1856e+01  -1.2038e+01

Columns 9373 to 9378
 -1.2038e+01  -1.1856e+01  -1.1856e+01  -1.2549e+01  -1.2261e+01  -1.1856e+01

Columns 9379 to 9384
 -1.1856e+01  -1.1856e+01  -1.1856e+01  -1.1856e+01  -1.1856e+01  -1.1856e+01

Columns 9385 to 9390
 -1.1856e+01  -1.1856e+01  -1.2038e+01  -1.1856e+01  -1.1856e+01  -1.1856e+01

Columns 9391 to 9396
 -1.1856e+01  -1.1856e+01  -1.2954e+01  -1.1856e+01  -1.1856e+01  -1.1856e+01

Columns 9397 to 9402
 -1.2954e+01  -1.2261e+01  -1.1856e+01  -1.1856e+01  -1.2549e+01  -1.1856e+01

Columns 9403 to 9408
 -1.2261e+01  -1.2038e+01  -1.1856e+01  -1.1856e+01  -1.1856e+01  -1.1856e+01

Columns 9409 to 9414
 -1.1856e+01  -1.2261e+01  -1.1856e+01  -1.2038e+01  -1.1856e+01  -1.2038e+01

Columns 9415 to 9420
 -1.2038e+01  -1.1856e+01  -1.1856e+01  -1.2261e+01  -1.2549e+01  -1.1856e+01

Columns 9421 to 9

to 9612
 -1.2038e+01  -1.2549e+01  -1.2038e+01  -1.2261e+01  -1.2038e+01  -1.2261e+01

Columns 9613 to 9618
 -1.2261e+01  -1.2038e+01  -1.2038e+01  -1.2038e+01  -1.2261e+01  -1.2038e+01

Columns 9619 to 9624
 -1.2038e+01  -1.2261e+01  -1.2261e+01  -1.2261e+01  -1.2038e+01  -1.2038e+01

Columns 9625 to 9630
 -1.2038e+01  -1.2549e+01  -1.2549e+01  -1.2038e+01  -1.2038e+01  -1.2038e+01

Columns 9631 to 9636
 -1.2261e+01  -1.2038e+01  -1.2261e+01  -1.2038e+01  -1.2038e+01  -1.2038e+01

Columns 9637 to 9642
 -1.2549e+01  -1.2549e+01  -1.2261e+01  -1.2038e+01  -1.2261e+01  -1.2038e+01

Columns 9643 to 9648
 -1.2261e+01  -1.2038e+01  -1.2038e+01  -1.2549e+01  -1.2038e+01  -1.2038e+01

Columns 9649 to 9654
 -1.2038e+01  -1.2038e+01  -1.2038e+01  -1.2038e+01  -1.2038e+01  -1.2038e+01

Columns 9655 to 9660
 -1.2038e+01  -1.2038e+01  -1.2549e+01  -1.2261e+01  -1.2261e+01  -1.2261e+01

Columns 9661 to 9666
 -1.2261e+01  -1.2261e+01  -1.2038e+01  -1.2038e+01  -1.2038e+01  -1.2261e+01

Columns 9667 

853 to 9858
 -1.2261e+01  -1.2038e+01  -1.2038e+01  -1.2038e+01  -1.2038e+01  -1.2038e+01

Columns 9859 to 9864
 -1.2038e+01  -1.2038e+01  -1.2038e+01  -1.2261e+01  -1.2549e+01  -1.2549e+01

Columns 9865 to 9870
 -1.2261e+01  -1.2549e+01  -1.2549e+01  -1.2261e+01  -1.2261e+01  -1.2261e+01

Columns 9871 to 9876
 -1.2261e+01  -1.2261e+01  -1.2261e+01  -1.2549e+01  -1.2261e+01  -1.2261e+01

Columns 9877 to 9882
 -1.2549e+01  -1.2954e+01  -1.2261e+01  -1.2261e+01  -1.2261e+01  -1.2261e+01

Columns 9883 to 9888
 -1.2261e+01  -1.2261e+01  -1.2261e+01  -1.2261e+01  -1.2261e+01  -1.2261e+01

Columns 9889 to 9894
 -1.2549e+01  -1.2261e+01  -1.2261e+01  -1.2261e+01  -1.2261e+01  -1.2261e+01

Columns 9895 to 9900
 -1.3648e+01  -1.2261e+01  -1.2549e+01  -1.2261e+01  -1.2261e+01  -1.2261e+01

Columns 9901 to 9906
 -1.2954e+01  -1.2261e+01  -1.2261e+01  -1.2261e+01  -1.2261e+01  -1.2261e+01

Columns 9907 to 9912
 -1.2261e+01  -1.2261e+01  -1.2549e+01  -1.2261e+01  -1.2261e+01  -1.2261e+01

Columns 9